In [2]:
import numpy as np
from transformers import BertTokenizer, BertModel
import torch

In [3]:
model_name = 'bert-base-uncased'
tokerniser = BertTokenizer.from_pretrained(model_name)

In [4]:
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
torch.cuda.is_available()

False

In [6]:
def preprocess_and_encode(text):

    sentences = text.split(". ")
    sentence_embeddings = []

    for sentence in sentences:
        tokens = tokerniser.encode_plus(
            sentence,
            add_special_tokens = True,
            truncation=True,
            max_length=128,
            padding='max_length',
            return_tensors='pt'
        )

        with torch.no_grad():
            outputs = model(**tokens)
            embeddings = outputs.last_hidden_state.squeeze(0).numpy()

            sentence_embeddings.append(embeddings)

    plot_embedding = np.mean(np.array(sentence_embeddings), axis=(0,1))

    return plot_embedding

In [7]:
sample = r"""In 1936, archaeologist Indiana Jones braves an ancient Peruvian temple filled with booby traps to retrieve a golden idol. Upon fleeing the temple, Indiana is confronted by rival archaeologist René Belloq and the indigenous Hovitos. Surrounded and outnumbered, Indiana is forced to surrender the idol to Belloq and escapes aboard a waiting Waco seaplane, in the process revealing his fear of snakes. Shortly after returning to the college in the United States where he teaches archaeology, Indiana is interviewed by two Army intelligence agents. They inform him that the Nazis, in their quest for occult power, are searching for his old mentor, Abner Ravenwood, who is the leading expert on the ancient Egyptian city of Tanis and possesses the headpiece of an artifact called the Staff of Ra. Indiana deduces that the Nazis are searching for Tanis because it is believed to be the location of the Ark of the Covenant, the biblical chest built by the Israelites to contain the fragments of the Ten Commandments; the Nazis believe that if they acquire it, their armies will become invincible. The Staff of Ra, meanwhile, is the key to finding the Well of Souls, a secret chamber in which the Ark is buried. The agents subsequently authorize Indiana to recover the Ark before the Nazis. Indiana travels to Nepal, only to find that Ravenwood has died and that the headpiece is in the possession of his daughter, Marion, Indiana's embittered former lover. Indiana offers to buy the headpiece for three thousand dollars, plus two thousand more when they return to the United States. Marion's tavern is suddenly raided by a group of thugs commanded by Nazi agent Toht. The tavern is burned down in the ensuing fight, during which Toht burns his hand on the searing hot headpiece as he tries to grab it. Indiana and Marion escape with the headpiece, with Marion declaring she will accompany Indiana in his search for the Ark so he can repay his debt. They travel to Cairo where they learn from Indiana's friend Sallah, a skilled excavator, that Belloq and the Nazis, led by Colonel Dietrich, are currently digging for the Well of Souls with a replica of the headpiece modeled after the scar on Toht's hand. In a bazaar, Nazi operatives attempt to kidnap Marion and as Indiana chases after them it appears that she dies in an explosion. While deciphering the markings on the headpiece, Indiana and Sallah realize that the Nazis have miscalculated the location of the Well of Souls. Using this to their advantage, they infiltrate the Nazi dig and use the Staff of Ra to determine the location correctly and uncover the Well of Souls, which is filled with snakes. Indiana fends off the snakes and acquires the Ark, but Belloq, Dietrich and the Nazis arrive to take it. They toss Marion into the well with Indiana and seal them in, but they manage to escape. After a fistfight with a giant Nazi mechanic, blowing up a flying wing on the airstrip, and chasing down a convoy of trucks, Indiana takes back the Ark before it can be shipped to Berlin. Indiana and Marion leave Cairo to escort the Ark to England on board a tramp steamer. The next morning, their boat is boarded by Belloq, Dietrich and the Nazis, who once again steal the Ark and kidnap Marion. Indiana stows away on their U-boat and follows them to an isolated island in the Aegean Sea where Belloq plans to test the power of the Ark before presenting it to Hitler. Indiana reveals himself and threatens to destroy the Ark with a rocket-propelled grenade launcher, but Belloq calls his bluff, knowing Indy cannot bear to eradicate an important historical artifact. Indiana surrenders and is tied to a post with Marion as Belloq performs a ceremonial opening of the Ark, which appears to contain nothing but sand. Suddenly, spirits resembling Old Testament Seraphim emerge from the Ark. Aware of the supernatural danger of looking at the opened Ark, Indiana warns Marion to close her eyes. The apparitions suddenly morph into "angels of death", and lightning bolts begin flying out of the Ark, gruesomely killing the Nazi soldiers, while Belloq, Dietrich and Toht meet even more gruesome fates. The fires rise into the sky, then fall back down to Earth and the Ark closes with a crack of thunder. Back in Washington, D.C., the Army intelligence agents tell a suspicious Indiana and Brody that the Ark "is someplace safe" to be studied by "top men". In reality, the Ark is sealed in a wooden crate labeled "top secret" and stored in a giant government warehouse filled with countless similar crates."""

In [8]:
encoded_embedding = preprocess_and_encode(sample)

In [9]:
encoded_embedding.shape

(768,)

In [10]:
with open('MovieSummaries/plot_summaries.txt') as file:
    movies = file.readlines()
    # print(file.readlines()[1].split("\t"))
    all_embeddings = []
    for movie in movies[:2]:
        uid, plot = movie.split("\t")
        embedding = preprocess_and_encode(plot)
    all_embeddings.append(embedding)   
all_embeddings = np.array(all_embeddings)     

In [11]:
import faiss

In [12]:
index = faiss.IndexFlatL2(encoded_embedding.shape[0])
index.add(all_embeddings)

In [13]:
def search(query_vector, index):
    k = 1  # Number of nearest neighbors to retrieve
    D, I = index.search(np.expand_dims(query_vector, axis=0), k)
    return I

In [15]:
query = "fighting nazis"
query_vector = preprocess_and_encode(query)


In [16]:
nei = search(query_vector, index)

: 

: 